### 朴素贝叶斯 文本分类
##### 数据集 ChnSentiCorp  中文-酒店评论
#### 描述： 共4000条酒店评论，positive-2000条，negtive-2000条
***
***
#### 训练集3800条 neg-1900条/pos-1900条
#### 测试集 200条 neg-100/pos-100

In [143]:
"""
数据集预处理并根据数据集生成词库
jieba分词
"""
import re
import jieba
import codecs
import os
import gc

#去掉数据集中的非中文字符
def clean_str(string):
    string = re.sub(r"[^\u4e00-\u9fff]", " ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip()

#深入数据集文件夹，进行数据清洗同时生成词库和训练集词数组
def initial_data(vocabBase0,comments0,labels0,pathroot):
    files = os.listdir(pathroot)
    vocabBase = vocabBase0
    comments = comments0
    labels = labels0
    for file in files:
        if os.path.isdir(pathroot+"/"+file):
            
            vocabBase,comments,labels = initial_data(vocabBase,comments,labels,pathroot+"/"+file)
        else:
            print('\r',"                                        ",end='')
            print('\r',"initial data:"+str(len(comments))+"/3800",end='')
            if file[0] == 'n':
                labels.append(0)
            else:
                labels.append(1)
            comment = ''
            f = codecs.open(pathroot+'/'+file,'r','utf-8',errors='ignore')
            for line in f:
                line = clean_str(line)
                comment += line
            f.close()
            comment=[word for word in jieba.cut(comment) if word.strip() != '']
            comments.append(comment) #使用原始的数据集词数组，未去重
            #emails.append(set(email))?
            vocabBase = vocabBase | set(comment) #set有去重功能，且排序
  
    return vocabBase,comments,labels


        
        
            
            


In [144]:
vocabBase = set([])
trainSet = []
label_list = []
pathroot = "F:/大三上/NB/data"
vocabBase,trainSet,label_list = initial_data(vocabBase,trainSet,label_list,pathroot) 

 initial data:3799/3800                  initial data:2195/3800                                        initial data:2728/3800

In [152]:
vocabBase

{'悬崖',
 '周边地区',
 '都行',
 '发癫',
 '烦人',
 '早睡',
 '挖',
 '日记本',
 '辉煌',
 '闹声',
 '拆开',
 '逃税',
 '不悦',
 '小睡',
 '幽默',
 '来晚',
 '糖',
 '窗房',
 '出现',
 '捣',
 '搜',
 '尾随',
 '轴流',
 '猴',
 '打搅',
 '眼花缭乱',
 '往下掉',
 '锁卡',
 '散步',
 '食饭',
 '旅游区',
 '联系电话',
 '地点',
 '房单',
 '恶言',
 '宴会厅',
 '坚决',
 '港岛',
 '过来',
 '说辞',
 '扎眼',
 '結果',
 '没多远',
 '皱巴巴',
 '真够',
 '太破',
 '层楼',
 '交换机',
 '外标',
 '小床',
 '算有',
 '旁邊',
 '修到',
 '原处',
 '国际会议中心',
 '完毕',
 '拎',
 '并非如此',
 '提醒',
 '桥',
 '部队',
 '果断',
 '开使',
 '报馆',
 '尖沙咀',
 '窗均',
 '这象',
 '一个样',
 '搬动',
 '这',
 '迷迷糊糊',
 '新婚旅行',
 '女友',
 '目的地',
 '承担',
 '足疗',
 '邻近',
 '维景',
 '小心',
 '还生',
 '噼里啪啦',
 '路面',
 '一路上',
 '风景区',
 '什',
 '千里眼',
 '黑到',
 '桑拿',
 '救',
 '下定',
 '故乡',
 '穿过',
 '房订',
 '地想',
 '不失',
 '卡式',
 '原先',
 '二是',
 '半山腰',
 '视频',
 '弥漫',
 '情形',
 '阳台门',
 '渗透',
 '宽频',
 '呕吐',
 '晕过去',
 '忽亮',
 '遺憾',
 '湖南路',
 '毛巾',
 '川菜',
 '简但',
 '可恨',
 '接线',
 '飞利浦',
 '很严',
 '首选',
 '玻璃房',
 '山上',
 '比小',
 '罗列',
 '尽管如此',
 '服务项目',
 '真可笑',
 '电掣',
 '磁卡',
 '无意',
 '何必',
 '另一边',
 '新城区',
 '留下',
 '大火',
 '前评',
 '涂鸦',
 '炒菜',
 '标准差',
 '柳州地区

In [145]:
from numpy import *


#生成词的0,1向量,统计词是否出现
def commentToVec(vocabBase,comment):
    rtnVec = [0]*len(vocabBase)
   # for word in vocabBase:
     #   rtnVec[vocabBase.index(word)] = 1
   # return rtnVec
    words = set(vocabBase) & set(comment)
    for word in words:
        rtnVec[vocabBase.index(word)] = 1
#     gc.collect()
    return rtnVec

#生成用于训练的0，1输入矩阵
def getTrainMatrix(vocabBase,trainSet):
    trainMatrix = []
    numComments = len(trainSet)
    i=0
    for comment in trainSet:
        print('\r',"                         ",end='')
        print('\r',"train: "+str(i)+"/"+str(numComments)+' ',end='')
        trainMatrix.append(commentToVec(list(vocabBase),comment))
        i=i+1
    return trainMatrix

#训练函数
def trainNB(commentMatrix,labelList):
    numComments = len(commentMatrix)
    sizeVocabBase = len(commentMatrix[0])
    
    #求正面评论出现的总概率
    #a = list(map(float,labelList))
    pPos = sum(labelList)/float(numComments)
    
    #初始化词向量，统计各词在两类评论中出现的总次数？同一评论中重复出现算1次
    ####使用拉普拉斯平滑，分子初始化为1，分母初始化为分类数
    p0Num = ones(sizeVocabBase)
    p1Num = ones(sizeVocabBase)
    
    p0Denom = 2.0 
    p1Denom = 2.0
    
    for i in range(numComments):
        print('\r',"                                    ",end='')
        print('\r',"NB: "+str(i)+"/"+str(numComments)+" ",end='')
        if label_list[i] == 1:
            p1Num += commentMatrix[i] #出现的词次数+1
            p1Denom += sum(commentMatrix[i])#分母加此邮件总词数？
        else:
            p0Num += commentMatrix[i]
            p0Denom += sum(commentMatrix[i])
            
    p1Vec = p1Num / p1Denom 
    p0Vec = p0Num / p0Denom
    return p0Vec,p1Vec,pPos
    
    

In [151]:
trainMatrix = getTrainMatrix(vocabBase,trainSet)
p0Vec,p1Vec,pPos = trainNB(trainMatrix,label_list)

 NB: 3799/3800                       3800                                                                            train: 2615/3800 train: 2796/3800                                                                         NB: 2452/3800 NB: 3573/3800 

In [148]:
from functools import reduce
"""
使用朴素贝叶斯
#为避免浮点数太小导致下溢，将乘法转为log的加法
"""
def prob(a,b):
    return a*b
def classifyNB(inputVec,p0Vec,p1Vec,pPos):
    L0 = inputVec*p0Vec
    L1 = inputVec*p1Vec
    #手动避免0概率
    i = 0
    for label in L0:
        if label == 0:
            L0[i] = 1
        i += 1
    i = 0
    for label in L1:
        if label == 0:
            L1[i] = 1
        i += 1
    p1 = sum(log(L1)) + log(pPos)
    p0 = sum(log(L0)) + log(1.0-pPos)
    #p1 = reduce(prob,L1)*pPos
    #p0 = reduce(prob,L0)*(1.0-pPos)

    if p1 > p0:
        return 1
    else:
        return 0
    
def testNB():
    noUse= set([])
    testSet = []
    testLabels=[]
    pathroot = "F:/大三上/NB/test"
    noUse,testSet,testLabels = initial_data(noUse,testSet,testLabels,pathroot)
    testMatrix = getTrainMatrix(vocabBase,testSet)
    i = 0
    TP=0
    FP=0
    TN=0
    FN=0
    predict=[]
    for testVec in testMatrix:
        result = classifyNB(testVec,p0Vec,p1Vec,pPos)
        if result == 1 and result == testLabels[i]:
            TP +=1
            predict.append(str(i)+" :TP")
        elif result == 1 and result != testLabels[i]:
            FP +=1
            predict.append(str(i)+" :FP")
        elif result != 1 and result == testLabels[i]:
            TN +=1
            predict.append(str(i)+" :TN")
        elif result != 1 and result != testLabels[i]:
            FN +=1
            predict.append(str(i)+" :FN")
        else:
            print(result,testLabels[i])
        i+=1
    Accurancy = (TP+TN)/(TP+TN+FP+FN)
    print("\nAccurancy:" + str(Accurancy))
    return testMatrix,predict,Accurancy

In [149]:
testMat,predict,accurancy=testNB()

 train: 199/200                          
Accurancy:0.855


### 结果：准确度 85.5%